# BIOMAG introduction material 2. - Pytorch basics

In [ ]:
import torch # pytorch basic package
from torch import nn # neural net 
from torch.utils.data import DataLoader, Dataset # to work with data
from torchvision import datasets # built-in data
from torchvision.transforms import ToTensor # to convert nparrays/images into pytorch tensors
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

First, let's check if the GPU is available:

In [ ]:
torch.cuda.is_available()

Now, let us download the datasets that we are going to use:

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

The PyTorch ``DataLoader`` acts as a wrapper over the ``Dataset`` and supports batching, random sampling, augmentation, etc., basically everything you would need for your neural network research.

In [ ]:
batch_size = 32

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

print(test_data[1][0].shape)

Let's see how a single sample from our test set looks like:

In [ ]:
print(f'{y[0] = }')
print(f'{y[0].item() = }')
print(f'{labels_map[y[0].item()] = }')
plt.imshow(X[0,0,:,:], cmap='gray')
plt.show()

Now let us create our own neural network in PyTorch. for that, we need to create our own class, which inherits most of the properties from the ``nn.Module`` class.

In [ ]:
# Set the GPU as the main device if it is available, if not, the CPU will suffice
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'We are using the following device: {device}')

In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    # here we define the neural net layers
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # this flattens out any tensor, meaning a 28x28 matrix will become a 1D vector of 784
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features = 28*28, out_features = 512, bias = True), # this will do the standard wx + b perceptron operation and will set all parameters to be the correct shape.
            nn.ReLU(), # perform the relu activation (z = relu(wx + b))
            nn.Linear(in_features = 512, out_features = 512), # and so on... note that in_features has to match the previous layer's out_features size.
            nn.ReLU(),
            nn.Linear(in_features = 512, out_features = 10) # bias is set True by default.
        )

    # here we specify how the data will flow through the network.
    def forward(self, x):
        # first we flatten our input x, which is an image (why is thid good/bad?)
        x = self.flatten(x)
        # then we perform the network prediction with the sequential layers specified in the init function.
        logits = self.linear_relu_stack(x)
        # we return the prediction (what is the size?)
        return logits

In [ ]:
# We create our model by instantiating the NeuralNetwork class and immediately move it to the GPU
model = NeuralNetwork().to(device)
print(model)

What happens if we apply our model on a batch of images from the training set?

In [ ]:
X, y = next(iter(train_dataloader))
result = model(X) # apply the operations specified in the forward function: it's the same as model.forward(X)

Now we get a problem, because our network (``model``) is on a different device compared to the samples and labels ``X`` and ``y``. Let's move our samples and labels to the GPU as well:

In [ ]:
X, y = X.to(device), y.to(device)
result = model(X)
print(f'{result.shape = }')

So how does a prediction look like for a single image?

In [ ]:
print(f'{result[0] = }')
print(f'{result = }')

These values are essentially probabilities, so for example, ``result[0]`` will output 10 probability values for the 1st image in the batch, each telling us how likely it is that the image is from class ``n`` (where ``n`` goes from 0  to 9). An easy way to tell immediately which class got the highest probability:

In [ ]:
print(result[0].argmax().item())
print(f' The predicted class for image 0 is: {labels_map[result[0].argmax().item()]}')


### Training (and testing) the network
Do train a network, we need a loss function and an optimizer method:

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
# training procedure for a single epoch
def train(train_dataloader, model, loss_fn, optimizer):
    num_samples = len(train_dataloader.dataset)
    num_batches = len(train_dataloader)
    model.train() # if we are in training mode, some functions will behave differently (e.g. dropout, batchnorm)

    training_loss = 0

    # iterate through the whole dataset, batch just stores the index of the batch.
    # if we have a total of 128 images, with a batch size of 32 we will have 4 batches before the for loop stops
    for batch, (X,y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)

        # calc yhat and loss value
        yhat = model(X)
        loss = loss_fn(yhat, y)
        training_loss += loss.item()

        # backpropagation: calculate the partial derivatives for each neural net parameter
        loss.backward()
        # update neural network parameters according to the partial derivatives
        optimizer.step()
        # zero out the variables that are used for storing the gradients
        optimizer.zero_grad()

        # we ususally don't want to print information at every batch, so make it a bit more scarce:
        if batch % 100 == 0:
            current_loss = loss.item()
            current = (batch + 1) * len(X) # len(X) is the batch size
            print(f'{current}/{num_samples} : {current_loss = }')
    
    training_loss /= num_batches
    return training_loss

We also write a test function to see how the model works on the test set:

In [ ]:
def test(epoch, test_dataloader, model, loss_fn):
    num_samples = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    model.eval() # if we are in eval mode, some functions behave differently
    test_loss = 0 
    correct = 0
    with torch.no_grad(): # we do not want to calculate gradients in eval mode
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y).item()
            test_loss += loss # we summarize the test loss across all batches
            correct_pred_locations = (pred.argmax(1) == y).type(torch.float) # also torch.argmax(pred, dim=1) is correct, and then we convert these locations to float
            correct += correct_pred_locations.sum().item() # we summarize how many items we got correct
    test_loss /= num_batches # we average the loss
    correct /= num_samples
    print(f'End of epoch {epoch+1}\n Accuracy: {(100*correct):.2f}%, Average loss: {test_loss:.4f}\n')
    return test_loss

Now let's do the actual training and evaluation:

In [ ]:
epochs = 5
train_losses = []
test_losses = []
for t in tqdm(range(epochs)):
    train_loss = train(train_dataloader, model, loss_fn, optimizer)
    test_loss = test(t, test_dataloader, model, loss_fn)

    train_losses.append(train_loss)
    test_losses.append(test_loss)
print("Done!")

Now, let us plot the training and validation curves:

In [ ]:
plt.plot(train_losses, label="training loss")
plt.plot(test_losses, label="test (validation) loss")
plt.legend()
plt.show()

Let us save this model for further use:

In [ ]:
torch.save(model.state_dict(), "mnist_starter_model.pth")

If we want to load it later:

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("mnist_starter_model.pth"))

Do a quick visual evaluation:

In [ ]:
model.eval()
indices = np.random.randint(len(test_data), size=(4)) # pick 4 random images from the test set
f = plt.figure()
for fig_idx, i in enumerate(indices):
    x, y = test_data[i][0], test_data[i][1]
    x = x.to(device)
    pred_vector = model(x)
    pred_value = labels_map[pred_vector.argmax().item()]
    gt_value = labels_map[y]
    plt.subplot(2,2,fig_idx+1)
    plt.title(f'GT: {gt_value}, pred: {pred_value}')
    plt.imshow(x.cpu()[0], cmap='gray')
    plt.axis('off')

plt.tight_layout()
plt.show()


## Homework
1. Try training the network for a longer time. What can you observe in terms of training and validation losses? Why do you think this is the case?
2. Try tweaking the network parameters (number of neurons, number of layers, etc) to reach a better accuracy than the original model after 10 epochs.